Hello, everyone. In this notebook, You can create all the dependency file and features to run my NN and LGB model.

NOTICE: 
1. I don't rerun the total approach, expecially i do not make sure filepath is correct. So you may need to check the input file path, but it should be easy.

2. The notebook has serval sub parts, which are independent to each other, you may want to copy them to seperated python script, and run every part, to fix some memory problem.

If there is any problem to reproduce, please contract me: liujilong(dot)me(at)gmail(dot)com

Let's start.

# Word2Vec Embedding

1. generate text file
2. run word2vec command line. (complied from google's source code)

In [ ]:
import logging
import time
import string
import pandas as pd

logging.basicConfig(level=logging.INFO)
use_cols = ['title', 'description']
file = '../input/fast_text.txt'


def load_text(path, f):
  print(f'Loading data from {path}...')
  tic = time.time()
  train2 = pd.read_csv(path, usecols=use_cols)
  train2['text'] = train2['title'].astype(str) + ' ' + train2['description'].astype(str)
  punct = string.punctuation.replace('|', '')+'\n'
  transtab = str.maketrans(dict.fromkeys(punct, ' '))

  for text in train2['text']:
    text = text.translate(transtab)
    f.write(text + '\n')
  f.flush()
  print('Done in {:.1f}s'.format(time.time() - tic))


if __name__ == '__main__':
  with open(file, mode='w', encoding='utf-8') as f:
    load_text('../input/test.csv', f)
    load_text('../input/train.csv', f)
    load_text('../input/train_active.csv', f)
    load_text('../input/test_active.csv', f)

In [ ]:
!word2vec -train fast_text.txt -output word2vec.bin -cbow 1 -size 200 -window 8 -negative 25 -hs 0 -sample 1e-4 -thread 12 -binary 1 -iter 15

# transform input from csv to pickle

I do this for fast data load.

In [ ]:
test = pd.read_csv('../input/test.csv', index_col="item_id",
                       parse_dates=["activation_date"])
print(test.shape)
test_filter = test[test['image'].notnull()]
print(test_filter.shape)
test_filter.to_pickle('../input/test_with_image.pickle')

test = pd.read_csv('../input/test.csv', index_col="item_id",
                       parse_dates=["activation_date"])
test.to_pickle('../input/test.pickle')

In [ ]:
img_id = ['4f029e2a00e892aa2cac27d98b52ef8b13d91471f613c8d3c38e3f29d4da0b0c', 
               '8513a91e55670c709069b5f85e12a59095b802877715903abef16b7a6f306e58', 
               '60d310a42e87cdf799afcd89dc1b11ae3fdc3d0233747ec7ef78d82c87002e83', 
               'b98b291bd04c3d92165ca515e00468fd9756af9a8f1df42505deed1dcfb5d7ae']

train = pd.read_csv('../input/train.csv', index_col="item_id",
                       parse_dates=["activation_date"])

for img in img_id:
    train.loc[train.image==img, 'image'] = float('nan')
    
train.to_pickle('../input/train.pickle')

train_filter = train[train['image'].notnull()]

train_filter.to_pickle('../input/train_with_image.pickle')

# Image Meta features

The code generates image meta features for train data. You should make small filepath changes to generate test iamge meta features

In [ ]:
import numpy as np
import pandas as pd
import time
from math import ceil
import cv2
import zipfile
from tqdm import tqdm
from keras.applications.vgg16 import VGG16, preprocess_input
from multiprocessing import Value, Pool
import gzip
from pathlib import PurePath
from scipy import sparse
from threading import Thread
from queue import Queue
import gc



def generate_files(n_items):
  print("Starting generate_files...")

  df =  pd.read_pickle('../input/train_with_image.pickle')
  #df =  pd.read_pickle('../input/test_with_image.pickle')
  ids = df['image'].tolist()
  n_items.value = len(ids)
  print("Total items:", n_items.value)

  for im_id in ids:
    if im_id == 'nan' or im_id == float('nan'):
      yield (im_id, None)
    img_file = '../input/train_jpg/{}.jpg'.format(im_id)
    #img_file = '../input/test_jpg/{}.jpg'.format(im_id)
    zbuf = np.asarray(bytearray(open(img_file, 'rb').read()), dtype="uint8")
    yield (im_id, zbuf)

  print("Finished generate_files")

def transform_image(img):
  yuv = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
  hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
  hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
  gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

  rgb_r = img[:, :, 0]
  rgb_g = img[:, :, 1]
  rgb_b = img[:, :, 2]
  yuv_y = yuv[:, :, 0]
  hsv_s = hsv[:, :, 1]
  hsv_v = hsv[:, :, 2]
  hls_l = hls[:, :, 1]

  res = []

  # Lightness
  res.append(np.mean(yuv_y))
  res.append(np.std(yuv_y))

  res.append(np.mean(hls_l))
  res.append(np.std(hls_l))

  # Saturation
  res.append(np.mean(hsv_s))
  res.append(np.std(hsv_s))

  # Colorfulness
  rgb_rg = rgb_r - rgb_g
  rgb_yb = (rgb_r + rgb_g) / 2 - rgb_b
  colorful = np.sqrt(np.var(rgb_rg) + np.var(rgb_yb)) + \
             0.3 * np.sqrt(np.mean(rgb_rg) ** 2 + np.mean(rgb_yb) ** 2)
  res.append(colorful)

  # Gray
  res.append(np.std(gray))

  # Color
  res.append(np.mean(rgb_r))
  res.append(np.mean(rgb_g))
  res.append(np.mean(rgb_b))

  # Shape
  shape = img.shape
  res.append(shape[0])
  res.append(shape[1])
  res.append(shape[0] * shape[1])

  # Blurrness
  res.append(cv2.Laplacian(gray, cv2.CV_64F).var())
  return res


def im_decode_resize(params):
  item_id, zbuf = params

  if zbuf is None:
    return item_id, None
  else:
    try:
      im = transform_image(cv2.imdecode(zbuf, cv2.IMREAD_COLOR))
    except Exception as e:
      print('Error decoding item_id', item_id, e)
      im = [0] * 15

    return item_id, im


# Based on https://gist.github.com/everilae/9697228
class ThreadedGenerator(object):
  """
  Generator that runs on a separate thread, returning values to calling
  thread. Care must be taken that the iterator does not mutate any shared
  variables referenced in the calling thread.
  """

  def __init__(self, iterator, queue_maxsize):
    self._iterator = iterator
    self._sentinel = object()
    self._queue = Queue(maxsize=queue_maxsize)
    self._thread = Thread(
      name=repr(iterator),
      target=self._run
    )

  def __repr__(self):
    return 'ThreadedGenerator({!r})'.format(self._iterator)

  def _run(self):
    try:
      for value in self._iterator:
        self._queue.put(value)

    finally:
      self._queue.put(self._sentinel)

  def __iter__(self):
    self._thread.start()
    for value in iter(self._queue.get, self._sentinel):
      yield value

    self._thread.join()


def get_shape(input, length):
  shape = getattr(input, "shape", None)
  if shape == None:
    shape = (len(input),)
  return (length,) + shape

if __name__ == '__main__':
  print("Loading model...")

  n_items = Value('i', -1)  # Async number of items
  pool = Pool(5)
  final = None
  bar = None
  try:
    start = time.time()
    count = 0
    generator = ThreadedGenerator(generate_files(n_items), 50)
    for index, (item_id, im) in enumerate(pool.imap(im_decode_resize, generator)):
      if bar is None:
        bar = tqdm(total=n_items.value, mininterval=10, unit_scale=True)
      if final is None:
        final = np.zeros(shape=get_shape(im, n_items.value))
      final[index] = im
      count += 1
      batch_size = 1000
      if count % batch_size == 0:
        bar.update(batch_size)


  finally:
    pool.close()
    del pool, bar

    gc.collect()

  np.save('../input/train_image_meta_wihtout_zero_size.npy', final)
  #np.save('../input/test_image_meta.npy', final)

# Image score features
ImageNet score and NIMA score for images. Also only for train, do some filepath changes for test.

In [ ]:
import numpy as np
import pandas as pd
import time
from math import ceil
import cv2
import zipfile
from tqdm import tqdm
from keras.applications.vgg16 import VGG16

from keras.preprocessing import image
import keras.applications.resnet50 as resnet50
import keras.applications.xception as xception
import keras.applications.inception_v3 as inception_v3
from multiprocessing import Value, Pool, Semaphore
from keras.applications.inception_resnet_v2 import InceptionResNetV2

from keras.models import Model
from keras.layers import Dense, Dropout
import gzip
from pathlib import PurePath
from scipy import sparse
from threading import Thread
from queue import Queue
import gc


def generate_files(n_items):
  print("Starting generate_files...")

  df = pd.read_pickle('../input/train_with_image.pickle')
  # df = df.iloc[:200]
  ids = df['image'].tolist()
  n_items.value = len(ids)
  print("Total items:", n_items.value)

  for im_id in ids:
    yield im_id
  print("Finished generate_files")


def decode_resize(im_id):
  img_file = '../../JPG_Avito/train_jpg/{}.jpg'.format(im_id)
  semaphore_1.acquire()
  img = cv2.imread(img_file)
  # if img is None:
  #   img = np.zeros(shape=(224, 224, 3))
  target_size = (224, 224)
  if img.size != target_size:
    img = cv2.resize(img, target_size, interpolation=cv2.INTER_CUBIC)
  return im_id, img


# Based on https://gist.github.com/everilae/9697228
class ThreadedGenerator(object):
  """
  Generator that runs on a separate thread, returning values to calling
  thread. Care must be taken that the iterator does not mutate any shared
  variables referenced in the calling thread.
  """

  def __init__(self, iterator, queue_maxsize):
    self._iterator = iterator
    self._sentinel = object()
    self._queue = Queue(maxsize=queue_maxsize)
    self._thread = Thread(
      name=repr(iterator),
      target=self._run
    )

  def __repr__(self):
    return 'ThreadedGenerator({!r})'.format(self._iterator)

  def _run(self):
    try:
      for value in self._iterator:
        self._queue.put(value)

    finally:
      self._queue.put(self._sentinel)

  def __iter__(self):
    self._thread.start()
    for value in iter(self._queue.get, self._sentinel):
      yield value

    self._thread.join()


def get_shape(input, length):
  shape = getattr(input, "shape", None)
  if shape is None:
    shape = (len(input),)
  return (length,) + shape

def get_nima_model():
  base_model = InceptionResNetV2(input_shape=(None, None, 3), include_top=False,
                                 pooling='avg', weights=None)
  x = Dropout(0.75)(base_model.output)
  x = Dense(10, activation='softmax')(x)

  model = Model(base_model.input, x)
  model.load_weights('../input/inception_resnet_weights.h5')
  return model


resnet_model = resnet50.ResNet50(weights='imagenet')
inception_model = inception_v3.InceptionV3(weights='imagenet')
xception_model = xception.Xception(weights='imagenet')
vgg16 = VGG16(weights='imagenet')
nima_model = get_nima_model()


def mean_score(scores):
    si = np.arange(1, 11, 1)
    mean = np.sum(scores * si, axis=1)
    return mean


def predict_batch(ori_batch):

  caffee_batch = resnet50.preprocess_input(np.array(ori_batch, dtype=np.float32))
  tf_batch = inception_v3.preprocess_input(np.array(ori_batch, dtype=np.float32))
  vgg_fea = vgg16.predict_on_batch(caffee_batch)
  resnet_fea = resnet_model.predict_on_batch(caffee_batch)
  incep_fea = inception_model.predict_on_batch(tf_batch)
  xcep_fea = xception_model.predict_on_batch(tf_batch)

  nima_fea = nima_model.predict_on_batch(tf_batch)
  mean = mean_score(nima_fea)


  res = np.max(vgg_fea, axis=1), np.max(resnet_fea, axis=1), np.max(incep_fea, axis=1), np.max(xcep_fea, axis=1), mean
  return np.stack(res, axis=1)

batch_size = 128
bar_iterval = 10  # in seconds

if __name__ == '__main__':
  print("Loading model...")

  semaphore_1 = Semaphore(2048)
  n_items = Value('i', -1)  # Async number of items
  pool = Pool(2)
  bar = None
  X_batch = []
  result = None
  total_size = 0
  try:
    # Threaded generator is usful for both parallel blocking read and to limit
    # items buffered by pool.imap (may cause OOM)
    generator = ThreadedGenerator(generate_files(n_items), 50)
    for item_id, im in pool.imap(decode_resize, generator):
      if bar is None:
        bar = tqdm(total=n_items.value, mininterval=bar_iterval,
                   unit_scale=True)

      X_batch.append(im)
      del im
      semaphore_1.release()

      if len(X_batch) == batch_size:
        batch_res = predict_batch(X_batch)
        if result is None:
          result = np.zeros(shape=get_shape(batch_res[0], n_items.value))
        result[total_size:total_size+batch_size] = batch_res
        total_size += batch_size
        del X_batch
        X_batch = []
        bar.update(batch_size)

    if len(X_batch) > 0:
        batch_res = predict_batch(X_batch)
        result[total_size:n_items.value] = batch_res
    bar.update(len(X_batch))

  finally:
    pool.close()
    del pool, X_batch

    if bar:
      bar.close()

    gc.collect()
    print(result.shape)
    np.save('../input/train_img_score.npy', result)


# Image numpy file

preprocess image files to one total numpy file, for fast load.

same for train-test file path change.

In [ ]:
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
from sys import getsizeof

from tqdm import tqdm

df = pd.read_csv('../input/train.csv', index_col="item_id",
                         parse_dates=["activation_date"])
img = df['image']
total_img = np.zeros(shape=(df.shape[0], 80, 60, 3), dtype=np.uint8)

for index, img_id in tqdm(enumerate(img)):
    img_file = '../input/train_jpg/{}.jpg'.format(img[index])
    iii = cv2.imread(img_file)
    if iii is not None:
        iii2 = cv2.resize(iii, (60, 80), interpolation=cv2.INTER_CUBIC)
        total_img[index] = iii2
        
np.save('train_img.npy', total_img)